In [1]:
import pandas as pd

import techminer.data as data

filepath = "../data/papers/urban-agriculture.csv"
df = pd.read_csv(filepath)
df = data.load_scopus(df)

2020-06-05 05:34:37,151 - INFO - Renaming and selecting columns ...
2020-06-05 05:34:42,952 - INFO - Formatting author names ...
2020-06-05 05:34:42,958 - INFO - Disambiguating author names ...
2020-06-05 05:34:43,035 - INFO - Removing part of titles in foreing languages ...
2020-06-05 05:34:43,038 - INFO - Fusioning author and index keywords ...
2020-06-05 05:34:43,051 - INFO - NumExpr defaulting to 8 threads.
2020-06-05 05:34:43,071 - INFO - Extracting countries from affiliations ...
2020-06-05 05:34:49,338 - INFO - Extracting institutions from affiliations ...
2020-06-05 05:34:49,353 - INFO - Extracting country of 1st author ...
2020-06-05 05:34:49,357 - INFO - Extracting affiliation of 1st author ...
2020-06-05 05:34:49,361 - INFO - Counting number of authors ...


In [ ]:
data.coverage(df)

### Keywords

### FindString Dialog Box pag. 69

### Cleaning a list pag. 102

### Fuzzy match editor pag.126

### Stemming And / Stemming Or

### APPS

In [ ]:
import techminer.by_year as year
year.app(df)

In [ ]:
import techminer.by_term as term
term.app(df)

In [ ]:
import techminer.by_term_per_year as term_year
term_year.app(df)

In [ ]:
import techminer.co_occurrence as co_occurrence
co_occurrence.app(df)

In [2]:
import techminer.correlation as correlation
correlation.app(df)

AppLayout(children=(HTML(value="<h1>Correlation Analysis</h1><hr style='height:2px;border-width:0;color:gray;b…

In [ ]:
import ipywidgets as widgets
from ipywidgets import Layout, AppLayout

In [ ]:

COLUMNS = [
    "Author Keywords",
    "Authors",
    "Countries",
    "Country 1st",
    "Document type",
    "Index Keywords",
    "Institution 1st",
    "Institutions",
    "Keywords",
    "Source title",
]

In [ ]:
FIGSIZE = (14, 10.0) ## Co-occurrente-analysis  <---- no tiene figura
FIGSIZE = (14, 9.0)

# NUEVA INTERFAZ

In [ ]:
from techminer.summary_by_term_per_year import documents_by_term_per_year, citations_by_term_per_year

In [ ]:
import techminer.plots as plt
from techminer.plots import COLORMAPS
from techminer.summary_by_term import citations_by_term, documents_by_term

WIDGET_WIDTH = "200px"
LEFT_PANEL_HEIGHT = "588px"
RIGHT_PANEL_WIDTH = "870px"
FIGSIZE = (14, 10.0)
PANE_HEIGHTS = ["80px", "650px", 0]

COLUMNS = [
    "Author Keywords",
    "Authors",
    "Countries",
    "Country 1st",
    "Document type",
    "Index Keywords",
    "Institution 1st",
    "Institutions",
    "Keywords",
    "Source title",
]



In [ ]:
#################
import ipywidgets as widgets
import numpy as np
from ipywidgets import AppLayout, Layout

from techminer.by_term import summary_by_term
from techminer.correlation import corr, corr_map
from techminer.maps import Map
from techminer.plots import COLORMAPS, chord_diagram

WIDGET_WIDTH = "200px"
LEFT_PANEL_HEIGHT = "650px"
RIGHT_PANEL_WIDTH = "870px"
FIGSIZE = (14, 10.0)
PANE_HEIGHTS = ["80px", "750px", 0]

COLUMNS = [
    "Author Keywords",
    "Authors",
    "Countries",
    "Country 1st",
    "Document type",
    "Index Keywords",
    "Institution 1st",
    "Institutions",
    "Keywords",
    "Source title",
]


#################


#
#
#  Correlation Analysis
#
#
def __body_0(x):
    #
    def server(**kwargs):
        #
        # Logic
        #
        column = kwargs["term"]
        by = kwargs["by"]
        method = kwargs["method"]
        min_link_value = float(kwargs["min_link_value"].split(' ')[0])
        cmap = kwargs["cmap"]
        filter_by = kwargs["filter_by"]
        filter_value = int(kwargs["filter_value"].split()[0])
        view = kwargs["view"]
        sort_by = kwargs["sort_by"]
        layout = kwargs["layout"]
        #
        #
        #
        s = summary_by_term(x, column)
        if filter_by == "Frequency":
            s = s[[column, "Num Documents"]]
            n_columns = len(s[s["Num Documents"] > filter_value])
            new_names = {
                a: "{} [{:d}]".format(a, b)
                for a, b in zip(s[column].tolist(), s["Num Documents"].tolist())
            }
        if filter_by == "Cited by":
            s = s[[column, "Cited by"]]
            n_columns = len(s[s["Cited by"] > filter_value])
            new_names = {
                a: "{} [{:d}]".format(a, b)
                for a, b in zip(s[column].tolist(), s["Cited by"].tolist())
            }
        #
        a = s[s.columns[1]].value_counts().sort_index(ascending=False)
        a = a.cumsum()
        a = a.sort_index(ascending=True)
        current_value = controls[4]["widget"].value
        controls[4]["widget"].options = [
            "{:d} [{:d}]".format(idx, w) for w, idx in zip(a, a.index)
        ]
        if current_value not in controls[4]["widget"].options:
            controls[4]["widget"].value = controls[4]["widget"].options[0]
        #
        #
        #
        if n_columns > 50:
            output.clear_output()
            with output:
                display(widgets.HTML("<h3>Matrix exceeds the maximum shape</h3>"))
                return
        #
        #
        #
        matrix = corr(
            x,
            column=column,
            by=by,
            method=method,
            # min_link_value=min_link_value,
            cmap=cmap,
            filter_by=filter_by,
            filter_value=filter_value,
            as_matrix=True,
            keywords=None,
        )
        #
        #
        #
        matrix = matrix.rename(columns=new_names, index=new_names)
        #
        output.clear_output()
        with output:
            if view == "Matrix":
                #
                # Sort order
                #
                if sort_by == "Frequency/Cited by asc.":
                    g = lambda m: int(m[m.find("[") + 1 : m.find("]")])
                    names = sorted(matrix.columns, key=g, reverse=False)
                    matrix = matrix.loc[names, names]
                if sort_by == "Frequency/Cited by desc.":
                    g = lambda m: int(m[m.find("[") + 1 : m.find("]")])
                    names = sorted(matrix.columns, key=g, reverse=True)
                    matrix = matrix.loc[names, names]
                if sort_by == "Alphabetic asc.":
                    matrix = matrix.sort_index(axis=0, ascending=True).sort_index(
                        axis=1, ascending=True
                    )
                if sort_by == "Alphabetic desc.":
                    matrix = matrix.sort_index(axis=0, ascending=False).sort_index(
                        axis=1, ascending=False
                    )
                #
                # View
                #
                display(
                    matrix.style.format(
                        lambda q: "{:+4.3f}".format(q) if q >= min_link_value else ""
                    ).background_gradient(cmap=cmap)
                )
                #
            if view == "Correlation map":
                #
                display(correlation_map(matrix=matrix, layout=layout, figsize=(10, 10), min_link_value=min_link_value))
                #
            if view == "Chord diagram":
                #
                display(chord_diagram(matrix, minval=min_link_value))
                #

    #
    # UI
    #
    controls = [
        {
            "arg": "term",
            "desc": "Term to analyze:",
            "widget": widgets.Dropdown(
                options=[z for z in COLUMNS if z in x.columns],
                ensure_option=True,
                disabled=False,
                layout=Layout(width=WIDGET_WIDTH),
            ),
        },
        {
            "arg": "by",
            "desc": "By Term:",
            "widget": widgets.Dropdown(
                options=[z for z in COLUMNS if z in x.columns],
                ensure_option=True,
                disabled=False,
                layout=Layout(width=WIDGET_WIDTH),
            ),
        },
        {
            "arg": "method",
            "desc": "Method:",
            "widget": widgets.Dropdown(
                options=["pearson", "kendall", "spearman"],
                ensure_option=True,
                disabled=False,
                continuous_update=True,
                layout=Layout(width=WIDGET_WIDTH),
            ),
        },
        {
            "arg": "filter_by",
            "desc": "Filter by:",
            "widget": widgets.Dropdown(
                options=["Frequency", "Cited by"],
                disable=False,
                layout=Layout(width=WIDGET_WIDTH),
            ),
        },
        {
            "arg": "filter_value",
            "desc": "Filter value:",
            "widget": widgets.Dropdown(
                options=[str(i) for i in range(10)],
                disable=False,
                layout=Layout(width=WIDGET_WIDTH),
            ),
        },
        {
            "arg": "min_link_value",
            "desc": "Min link value:",
            "widget": widgets.Dropdown(
                options="-1.00 -0.25 0.00 0.125 0.250 0.375 0.500 0.625 0.750 0.875".split(
                    " "
                ),
                ensure_option=True,
                disabled=False,
                continuous_update=True,
                layout=Layout(width=WIDGET_WIDTH),
            ),
        },
        {
            "arg": "view",
            "desc": "View:",
            "widget": widgets.Dropdown(
                options=["Matrix", "Correlation map", "Chord diagram"],
                ensure_option=True,
                disabled=False,
                continuous_update=True,
                layout=Layout(width=WIDGET_WIDTH),
            ),
        },
        {
            "arg": "cmap",
            "desc": "Matrix colormap:",
            "widget": widgets.Dropdown(
                options=COLORMAPS, disable=False, layout=Layout(width=WIDGET_WIDTH),
            ),
        },
        {
            "arg": "sort_by",
            "desc": "Sort order:",
            "widget": widgets.Dropdown(
                options=[
                    "Alphabetic asc.",
                    "Alphabetic desc.",
                    "Frequency/Cited by asc.",
                    "Frequency/Cited by desc.",
                ],
                disable=False,
                layout=Layout(width=WIDGET_WIDTH),
            ),
        },
        {
            "arg": "layout",
            "desc": "Map layout:",
            "widget": widgets.Dropdown(
                options=[
                    "Circular",
                    "Kamada Kawai",
                    "Planar",
                    "Random",
                    "Spectral",
                    "Spring",
                    "Shell",
                ],
                disable=False,
                layout=Layout(width=WIDGET_WIDTH),
            ),
        },
    ]
    #
    args = {control["arg"]: control["widget"] for control in controls}
    output = widgets.Output()
    with output:
        display(widgets.interactive_output(server, args,))
    return widgets.HBox(
        [
            widgets.VBox(
                [
                    widgets.VBox(
                        [widgets.Label(value=control["desc"]), control["widget"]]
                    )
                    for control in controls
                ],
                layout=Layout(height=LEFT_PANEL_HEIGHT, border="1px solid gray"),
            ),
            widgets.VBox([output], layout=Layout(width=RIGHT_PANEL_WIDTH, align_items="center")),
        ]
    )


#
#
# APP
#
#


def app(df):
    #
    body = widgets.Tab()
    body.children = [__body_0(df)]
    body.set_title(0, "Matrix")
    #
    return AppLayout(
        header=widgets.HTML(
            value="<h1>{}</h1><hr style='height:2px;border-width:0;color:gray;background-color:gray'>".format(
                "Correlation Analysis"
            )
        ),
        center=body,
        pane_heights=PANE_HEIGHTS,
    )


#####
app(df)

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx


def correlation_map(matrix, layout="Kamada Kawai", figsize=(11, 11), min_link_value=0):
    """Computes the correlation map directly using networkx.
    """

    if len(matrix.columns) > 50:
        return "Maximum number of nodex exceded!"

    #
    # Data preparation
    #
    terms = matrix.columns.tolist()

    n = len(matrix.columns)
    edges_75 = []
    edges_50 = []
    edges_25 = []
    other_edges = []

    for icol in range(n):
        for irow in range(icol + 1, n):
            if (
                min_link_value is None
                or matrix[terms[icol]][terms[irow]] >= min_link_value
            ):
                if matrix[terms[icol]][terms[irow]] > 0.75:
                    edges_75.append((terms[icol], terms[irow]))
                elif matrix[terms[icol]][terms[irow]] > 0.50:
                    edges_50.append((terms[icol], terms[irow]))
                elif matrix[terms[icol]][terms[irow]] > 0.25:
                    edges_25.append((terms[icol], terms[irow]))
                elif matrix[terms[icol]][terms[irow]] > 0.0:
                    other_edges.append((terms[icol], terms[irow]))

    if len(edges_75) == 0:
        edges_75 = None
    if len(edges_50) == 0:
        edges_50 = None
    if len(edges_25) == 0:
        edges_25 = None
    if len(other_edges) == 0:
        other_edges = None

    #
    # Network drawing
    #
    fig = plt.Figure(figsize=figsize)
    ax = fig.subplots()
    #
    draw_dict = {
        "Circular": nx.draw_circular,
        "Kamada Kawai": nx.draw_kamada_kawai,
        "Planar": nx.draw_planar,
        "Random": nx.draw_random,
        "Spectral": nx.draw_spectral,
        "Spring": nx.draw_spring,
        "Shell": nx.draw_shell,
    }
    draw = draw_dict[layout]

    G = nx.Graph(ax=ax)
    G.clear()
    #
    G.add_nodes_from(terms)
    #
    if edges_75 is not None:
        G.add_edges_from(edges_75)
    if edges_50 is not None:
        G.add_edges_from(edges_50)
    if edges_25 is not None:
        G.add_edges_from(edges_25)
    if other_edges is not None:
        G.add_edges_from(other_edges)
    #
    with_labels = True
    if edges_75 is not None:
        draw(
            G,
            ax=ax,
            edgelist=edges_75,
            width=3,
            edge_color="k",
            with_labels=with_labels,
            font_weight="bold",
            node_color='lightgray',
            node_size = 1,
            bbox=dict(facecolor='white', alpha=1.0),
        )
        with_labels = False
    #
    if edges_50 is not None:
        draw(
            G,
            ax=ax,
            edgelist=edges_50,
            edge_color="k",
            width=1,
            with_labels=with_labels,
            font_weight=with_labels,
            node_color='lightgray',
            node_size = 1,
            bbox=dict(facecolor='white', alpha=1.0),
        )
        with_labels = False
    #
    if edges_25 is not None:
        draw(
            G,
            ax=ax,
            edgelist=edges_25,
            edge_color="red",
            width=1,
            style="dashed",
            alpha=1.0,
            with_labels=with_labels,
            font_weight=with_labels,
            node_color='lightgray',
            node_size = 1,
            bbox=dict(facecolor='white', alpha=1.0),
        )
        with_labels = False
    if other_edges is not None:
        draw(
            G,
            ax=ax,
            edgelist=other_edges,
            edge_color="red",
            width=1,
            alpha=1.0,
            style="dotted",
            with_labels=with_labels,
            font_weight=with_labels,
            node_color='lightgray',
            node_size = 1,
            bbox=dict(facecolor='white', alpha=1.0),
        )
        with_labels = False

    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    ax.set_xlim(
        xlim[0] - 0.15 * (xlim[1] - xlim[0]), xlim[1] + 0.15 * (xlim[1] - xlim[0])
    )
    ax.set_ylim(
        ylim[0] - 0.15 * (ylim[1] - ylim[0]), ylim[1] + 0.15 * (ylim[1] - ylim[0])
    )
    return fig


matrix = pd.DataFrame(
    [
        [1.00, 0.70, 0.26, 0.70, 1.00, 0.20],
        [0.70, 1.00, 0.20, 1.00, 0.70, 0.26],
        [0.26, 0.20, 1.00, 0.70, 1.00, 0.20],
        [1.00, 0.70, 0.26, 0.70, 1.00, 0.20],
        [0.70, 1.00, 0.20, 1.00, 0.70, 0.26],
        [0.26, 0.20, 1.00, 0.70, 1.00, 0.20],
    ],
    columns=list("abcdef"),
    index=list("abcdef"),
)

correlation_map(matrix, figsize=(10, 10), min_link_value=0)

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
interact(slow_function,i=widgets.FloatSlider(min=1e5, max=1e7, step=1e5, continuous_update=True));

In [ ]:
def slow_function(i):
    print(int(i),list(x for x in range(int(i)) if
                str(x)==str(x)[::-1] and
                str(x**2)==str(x**2)[::-1]))
    return

In [ ]:
x = "hola"
x[:-1]

In [ ]:
import pandas as pd
x = pd.DataFrame({'a': [1, 2, 3, 4, 5], 'b': list('ABCDE')})

x.values.ravel().tolist()

In [ ]:
x

In [ ]:
x.values.shape

In [ ]:
print('hola')

In [ ]:
m = "abcdefg [1000]"
m[m.find('[')+1:m.find(']')]


In [ ]:
import networkx as nx
G = nx.petersen_graph()
nx.draw(G, with_labels=True, font_weight='bold')

In [ ]:
nx.draw_shell(G, nlist=[range(5, 10), range(5)], with_labels=True, font_weight='bold')

In [ ]:
nx.draw_circular(G, nlist=[range(5, 10), range(5)], with_labels=True, font_weight='bold')

In [ ]:
nx.draw_kamada_kawai(G, nlist=[range(5, 10), range(5)], with_labels=True, font_weight='bold')

In [ ]:
nx.draw_spectral(G, nlist=[range(5, 10), range(5)], with_labels=True, font_weight='bold')

In [ ]:
nx.draw_spring(G, nlist=[range(5, 10), range(5)], with_labels=True, font_weight='bold')

In [ ]:
nx.draw_shell(G, nlist=[range(5, 10), range(5)], with_labels=True, font_weight='bold')